In [2]:
import geopandas
import pandas
from pyproj import Geod

In [48]:
hazard = "storm"
extract_regex = "epoch:(?P<epoch>[^|]*)\|hazard:(?P<hazard>[^|]*)\|quantile:(?P<quantile>[^|]*)\|ssp:(?P<ssp>[^|]*)"
extract_colnames = ["epoch", "quantile", "ssp"]
threshold = 1

In [4]:
geod = Geod(ellps="WGS84")

In [49]:
wide_df = geopandas.read_parquet(f"exposure/Supply network_link.{hazard}.parquet")
wide_df["length_m"] = wide_df.geometry.apply(geod.geometry_length).astype("int")

In [ ]:
# datacols = [c for c in wide_df.columns if ":" in c]
# keepcols = [
#     "NO",
#     "FROMNODENO",
#     "TONODENO",
#     "split",
#     "length_m",
# ]
# melted = wide_df.melt(id_vars=keepcols, value_vars=datacols)
# hazard_params = melted.variable.str.extract(extract_regex)

# long_df = (
#     pandas.concat([melted, hazard_params], axis=1)
#     .drop("variable", axis=1)
#     .query("value > 0")
# )

In [7]:
# indexcols = keepcols + extract_colnames
# long_neat = long_df.pivot(
#     index=indexcols,
#     columns="hazard",
#     values="value",
# ).fillna(0)

In [8]:
# long_neat.to_csv(f"exposure/Supply network_link.{hazard}.csv")

## Join region IDs

In [50]:
lookup = wide_df[['NO','FROMNODENO','TONODENO', 'split', 'length_m', 'geometry']]

In [51]:
adm1 = geopandas.read_file("gadm.gpkg")[["GID_0", "NAME_0", "GID_1", "NAME_1", "geometry"]]

In [52]:
zones = geopandas.read_file("network_shp/Supply network_zone.SHP").rename(columns={'NO':'ZONE_NO'})

In [53]:
lookup_adm1 = (
    geopandas
    .sjoin(lookup, adm1, how="left", predicate="intersects")
    .drop(columns='index_right')
)

In [54]:
lookup_adm1_zone = (
    geopandas
    .sjoin(lookup_adm1, zones, how="left", predicate="intersects")
    .drop(columns='index_right')
)
lookup_adm1_zone.head()

,NO,FROMNODENO,TONODENO,split,length_m,geometry,GID_0,NAME_0,GID_1,NAME_1,ZONE_NO
0,1,1,2,0,191,"LINESTRING (25.55166 -33.94136, 25.55179 -33.9...",ZAF,South Africa,ZAF.1_1,Eastern Cape,67.0
1,1,2,1,0,191,"LINESTRING (25.55233 -33.94300, 25.55217 -33.9...",ZAF,South Africa,ZAF.1_1,Eastern Cape,67.0
2,2,3,4,0,44,"LINESTRING (25.55008 -33.93951, 25.55008 -33.9...",ZAF,South Africa,ZAF.1_1,Eastern Cape,67.0
3,2,4,3,0,44,"LINESTRING (25.55040 -33.93925, 25.55022 -33.9...",ZAF,South Africa,ZAF.1_1,Eastern Cape,67.0
4,3,5,6,0,607,"LINESTRING (25.58222 -33.98484, 25.58297 -33.9...",ZAF,South Africa,ZAF.1_1,Eastern Cape,67.0


In [55]:
lookup_adm1_zone.to_parquet(f"exposure/Supply network_link.{hazard}.splits.parquet")

In [56]:
lookup_adm1_zone.to_file(f"exposure/Supply network_link.{hazard}.splits.gpkg")

In [57]:
lookup_adm1_zone[["NO","FROMNODENO","TONODENO","split","length_m","GID_0","NAME_0","GID_1","NAME_1","ZONE_NO"]] \
    .to_csv(f"exposure/Supply network_link.{hazard}.splits.csv")

## Summarise by ADM1

In [32]:
lookup_adm1.set_index(['NO','FROMNODENO','TONODENO', 'split'], inplace=True)

In [33]:
above_threshold_df = long_df.query(f"value > {threshold}")

NameError: name 'long_df' is not defined

In [24]:
above_threshold_df = (
    above_threshold_df
    .set_index(['NO','FROMNODENO','TONODENO', 'split'])
    .join(lookup_adm1, how="left")
)

In [25]:
above_threshold_df.head()

,FROMNODENO,TONODENO,split,length_m,value,epoch,hazard,quantile,ssp,GID_0,NAME_0,GID_1,NAME_1
NO,,,,,,,,,,,,,
1,1,2,0,191,1.25,2040-2059,hd35,p90,ssp126,ZAF,South Africa,ZAF.1_1,Eastern Cape
1,2,1,0,191,1.25,2040-2059,hd35,p90,ssp126,ZAF,South Africa,ZAF.1_1,Eastern Cape
1,1,2,0,191,1.80,2040-2059,hd35,p90,ssp245,ZAF,South Africa,ZAF.1_1,Eastern Cape
1,2,1,0,191,1.80,2040-2059,hd35,p90,ssp245,ZAF,South Africa,ZAF.1_1,Eastern Cape
1,1,2,0,191,2.00,2040-2059,hd35,p90,ssp585,ZAF,South Africa,ZAF.1_1,Eastern Cape


In [27]:
indexcols_summary = [
    "NO",
    "FROMNODENO",
    "TONODENO",
    "split",
    "GID_0",
    "NAME_0",
    "GID_1",
    "NAME_1",
] + extract_colnames

above_threshold_df_neat = above_threshold_df.reset_index().pivot(
    index=indexcols_summary,
    columns="hazard",
    values="length_m",
).fillna(0)
above_threshold_df_neat.head()

hazard                                                                                           hd35  \
NO FROMNODENO TONODENO split GID_0 NAME_0       GID_1   NAME_1       epoch     quantile ssp             
1  1          2        0     ZAF   South Africa ZAF.1_1 Eastern Cape 2040-2059 p90      ssp126  191.0   
                                                                                        ssp245  191.0   
                                                                                        ssp585  191.0   
   2          1        0     ZAF   South Africa ZAF.1_1 Eastern Cape 2040-2059 p90      ssp126  191.0   
                                                                                        ssp245  191.0   

hazard                                                                                          hd42  \
NO FROMNODENO TONODENO split GID_0 NAME_0       GID_1   NAME_1       epoch     quantile ssp            
1  1          2        0     ZAF   South Africa ZAF.1_1 Eastern Cape 2040-2059 p90      ssp126   0.0   
                                                                                        ssp245   0.0   
                                                                                        ssp585   0.0   
   2          1        0     ZAF   South Africa ZAF.1_1 Eastern Cape 2040-2059 p90      ssp126   0.0   
                                                                                        ssp245   0.0   

hazard                                                                                          hd45  
NO FROMNODENO TONODENO split GID_0 NAME_0       GID_1   NAME_1       epoch     quantile ssp           
1  1          2        0     ZAF   South Africa ZAF.1_1 Eastern Cape 2040-2059 p90      ssp126   0.0  
                                                                                        ssp245   0.0  
                                                                                        ssp585   0.0  
   2          1        0     ZAF   South Africa ZAF.1_1 Eastern Cape 2040-2059 p90      ssp126   0.0  
                                                                                        ssp245   0.0

In [30]:
groupby_cols = [
    "GID_0",
    "NAME_0",
    "GID_1",
    "NAME_1",
] + extract_colnames
summary = (
    above_threshold_df_neat
    .reset_index()
    .drop(columns=["NO", "FROMNODENO", "TONODENO", "split"])
    .groupby(groupby_cols)
    .sum()
)
summary

hazard                                                                         hd35  \
GID_0 NAME_0   GID_1    NAME_1             epoch     quantile ssp                     
AGO   Angola   AGO.10_1 Huíla              1995-2014 median   historical  1086716.0   
                                                     p10      historical   587790.0   
                                                     p90      historical  2908802.0   
                                           2040-2059 median   ssp126      2167244.0   
                                                              ssp245      2406440.0   
...                                                                             ...   
ZWE   Zimbabwe ZWE.9_1  Matabeleland South 2040-2059 p10      ssp245      2762296.0   
                                                              ssp585      2762296.0   
                                                     p90      ssp126      2762296.0   
                                                              ssp245      2762296.0   
                                                              ssp585      2762296.0   

hazard                                                                         hd42  \
GID_0 NAME_0   GID_1    NAME_1             epoch     quantile ssp                     
AGO   Angola   AGO.10_1 Huíla              1995-2014 median   historical        0.0   
                                                     p10      historical        0.0   
                                                     p90      historical        0.0   
                                           2040-2059 median   ssp126            0.0   
                                                              ssp245            0.0   
...                                                                             ...   
ZWE   Zimbabwe ZWE.9_1  Matabeleland South 2040-2059 p10      ssp245       758802.0   
                                                              ssp585       823312.0   
                                                     p90      ssp126      2762296.0   
                                                              ssp245      2762296.0   
                                                              ssp585      2762296.0   

hazard                                                                         hd45  
GID_0 NAME_0   GID_1    NAME_1             epoch     quantile ssp                    
AGO   Angola   AGO.10_1 Huíla              1995-2014 median   historical        0.0  
                                                     p10      historical        0.0  
                                                     p90      historical        0.0  
                                           2040-2059 median   ssp126            0.0  
                                                              ssp245            0.0  
...                                                                             ...  
ZWE   Zimbabwe ZWE.9_1  Matabeleland South 2040-2059 p10      ssp245            0.0  
                                                              ssp585            0.0  
                                                     p90      ssp126      1855732.0  
                                                              ssp245      1855732.0  
                                                              ssp585      2762296.0  

[2887 rows x 3 columns]

In [31]:
summary.to_csv(f"exposure/admin1.{hazard}.csv")

In [32]:
summary_wide = (
    summary
    .reset_index()
    .pivot(
        index=[
            "GID_0",
            "NAME_0",
            "GID_1",
            "NAME_1",
        ],
        columns=extract_colnames,
    )
    .reset_index()
    .set_index("GID_1")
)
summary_wide

hazard   GID_0    NAME_0              NAME_1       hd35                        \
epoch                                         1995-2014                         
quantile                                         median        p10        p90   
ssp                                          historical historical historical   
GID_1                                                                           
AGO.10_1   AGO    Angola               Huíla  1086716.0   587790.0  2908802.0   
AGO.11_1   AGO    Angola              Luanda        NaN        NaN    58146.0   
AGO.12_1   AGO    Angola         Lunda Norte   379186.0   297342.0   379186.0   
AGO.13_1   AGO    Angola           Lunda Sul  1591162.0   793590.0  1591162.0   
AGO.14_1   AGO    Angola             Malanje   511126.0   115704.0   910594.0   
...        ...       ...                 ...        ...        ...        ...   
ZWE.5_1    ZWE  Zimbabwe    Mashonaland East  2679838.0  2679838.0  2679838.0   
ZWE.6_1    ZWE  Zimbabwe    Mashonaland West  1930730.0  1930730.0  1930730.0   
ZWE.7_1    ZWE  Zimbabwe            Masvingo  3330634.0  3330634.0  3330634.0   
ZWE.8_1    ZWE  Zimbabwe  Matabeleland North  2320642.0  2320642.0  2320642.0   
ZWE.9_1    ZWE  Zimbabwe  Matabeleland South  2762296.0  2762296.0  2762296.0   

hazard                                                ...       hd45  \
epoch     2040-2059                                   ...  1995-2014   
quantile     median                              p10  ...        p90   
ssp          ssp126     ssp245     ssp585     ssp126  ... historical   
GID_1                                                 ...              
AGO.10_1  2167244.0  2406440.0  2962964.0  1086716.0  ...        0.0   
AGO.11_1        NaN    58146.0    58146.0        NaN  ...        0.0   
AGO.12_1   379186.0   379186.0   379186.0   297342.0  ...        0.0   
AGO.13_1  1591162.0  1591162.0  1591162.0  1591162.0  ...        0.0   
AGO.14_1   910594.0   910594.0  1193946.0   180044.0  ...        0.0   
...             ...        ...        ...        ...  ...        ...   
ZWE.5_1   2679838.0  2679838.0  2679838.0  2679838.0  ...        0.0   
ZWE.6_1   1930730.0  1930730.0  1930730.0  1930730.0  ...        0.0   
ZWE.7_1   3330634.0  3330634.0  3330634.0  3330634.0  ...  1472184.0   
ZWE.8_1   2320642.0  2320642.0  2320642.0  2320642.0  ...        0.0   
ZWE.9_1   2762296.0  2762296.0  2762296.0  2762296.0  ...  1001180.0   

hazard                                                                   \
epoch    2040-2059                                                        
quantile    median                         p10                      p90   
ssp         ssp126    ssp245     ssp585 ssp126 ssp245 ssp585     ssp126   
GID_1                                                                     
AGO.10_1       0.0       0.0        0.0    0.0    0.0    0.0        0.0   
AGO.11_1       NaN       0.0        0.0    NaN    NaN    NaN        0.0   
AGO.12_1       0.0       0.0        0.0    0.0    0.0    0.0        0.0   
AGO.13_1       0.0       0.0        0.0    0.0    0.0    0.0        0.0   
AGO.14_1       0.0       0.0        0.0    0.0    0.0    0.0        0.0   
...            ...       ...        ...    ...    ...    ...        ...   
ZWE.5_1        0.0       0.0        0.0    0.0    0.0    0.0   183328.0   
ZWE.6_1        0.0       0.0        0.0    0.0    0.0    0.0   370116.0   
ZWE.7_1   158948.0  158948.0  1206188.0    0.0    0.0    0.0  2331458.0   
ZWE.8_1        0.0       0.0        0.0    0.0    0.0    0.0   818902.0   
ZWE.9_1   114126.0  758802.0   823312.0    0.0    0.0    0.0  1855732.0   

hazard                          
epoch                           
quantile                        
ssp          ssp245     ssp585  
GID_1                           
AGO.10_1        0.0        0.0  
AGO.11_1        0.0        0.0  
AGO.12_1        0.0        0.0  
AGO.13_1        0.0        0.0  
AGO.14_1        0.0        0.0  
...             ...

In [39]:
newcols = []
for c in summary_wide.columns:
    parts = [p for p in c if p != '']
    newcols.append("_".join(parts))
summary_wide.columns = newcols

In [41]:
summary_gdf = adm1.set_index("GID_1").join(summary_wide.drop(columns=['GID_0', 'NAME_0', 'NAME_1'])).fillna(0)
summary_gdf.head()

,GID_0,NAME_0,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry,...,hd45_1995-2014_p90_historical,hd45_2040-2059_median_ssp126,hd45_2040-2059_median_ssp245,hd45_2040-2059_median_ssp585,hd45_2040-2059_p10_ssp126,hd45_2040-2059_p10_ssp245,hd45_2040-2059_p10_ssp585,hd45_2040-2059_p90_ssp126,hd45_2040-2059_p90_ssp245,hd45_2040-2059_p90_ssp585
GID_1,,,,,,,,,,,,,,,,,,,,,
AGO.1_1,AGO,Angola,Bengo,0,0,Província,Province,1,AO.BO,"MULTIPOLYGON (((13.12708 -7.83736, 13.12708 -7...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AGO.2_1,AGO,Angola,Benguela,Benguella,0,Província,Province,2,AO.BG,"MULTIPOLYGON (((12.52236 -13.45208, 12.52236 -...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AGO.3_1,AGO,Angola,Bié,0,0,Província,Province,3,AO.BI,"MULTIPOLYGON (((17.80354 -13.74234, 17.81293 -...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AGO.4_1,AGO,Angola,Cabinda,0,0,Província,Province,4,AO.CB,"MULTIPOLYGON (((12.21127 -5.78652, 12.21125 -5...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AGO.5_1,AGO,Angola,Cuando Cubango,0,0,Província,Province,5,AO.CC,"MULTIPOLYGON (((19.89892 -17.87674, 19.89082 -...",...,863820.0,0.0,0.0,0.0,0.0,0.0,0.0,1536928.0,1767966.0,2211058.0


In [43]:
summary_gdf.to_file(f"exposure/admin1.{hazard}.gpkg")